# Automating Non-GAAP Metric Extraction from 8-Ks

**Adjusted EBITDA, Free Cash Flow, and Core Earnings** are central to financial analysis—often more informative than GAAP figures. But extracting these metrics from earnings releases and 8-Ks is difficult. Formats are inconsistent, terminology varies, and key tables are often buried deep in unstructured disclosures.

**Traditional tools** (regex, templates, table parsers) struggle with this inconsistency, making extraction slow, manual, and error-prone.

Captide uses a **retrieval-augmented generation (RAG) API** to pull metrics like Adjusted EBITDA from SEC filings. It returns structured, schema-consistent JSON—even from messy source documents—making the data ready for modeling or analysis.

We first collect recent 8-Ks (Item 2.02) using a `fetch_documents` function. Then we send a structured prompt to Captide to extract Net Income to Adjusted EBITDA reconciliations.

In [15]:
import os

CAPTIDE_API_KEY = os.getenv("CAPTIDE_API_KEY")

HEADERS = {
    "X-API-Key": CAPTIDE_API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

TICKERS = ["LYFT", "PLTR", "UBER"]

In [16]:
import re, json, requests, pandas as pd
from typing import Dict, List

def is_valid_fiscal_period(fp: str) -> bool:
    m = re.match(r"Q([1-4]) (\d{4})", fp)
    return bool(m and int(m.group(2)) > 2022)

def is_valid_document(doc: Dict) -> bool:
    if doc["sourceType"] == "8-K":
        return "2.02" in doc.get("additionalKwargs", {}).get("item", "")
    return True

def fetch_documents(ticker: str) -> List[Dict]:
    url = f"https://rest-api.captide.co/api/v1/companies/ticker/{ticker}/documents"
    docs = requests.get(url, headers=HEADERS, timeout=60).json()
    return [
        {"ticker": doc["ticker"],
         "fiscalPeriod": doc["fiscalPeriod"],
         "sourceLink": doc["sourceLink"]}
        for doc in docs
        if doc["sourceType"] == "8-K"
        and "fiscalPeriod" in doc
        and is_valid_fiscal_period(doc["fiscalPeriod"])
        and is_valid_document(doc)
    ]

In [17]:
def parse_sse_response(sse_text: str) -> Dict:
    try:
        lines = [l[6:] for l in sse_text.splitlines() if l.startswith("data: ")]
        for l in lines:
            obj = json.loads(l)
            if obj.get("type") == "full_answer":
                content = re.sub(r"\s*\[#\w+\]", "", obj["content"])
                m = re.search(r"\{.*\}", content, re.DOTALL)
                return json.loads(m.group(0)) if m else {}
    except Exception:
        pass
    return {}

def fetch_metrics_with_prompt(source_links: List[str], prompt: str) -> Dict:
    payload = {"query": prompt, "sourceLink": source_links}
    r = requests.post(
        "https://rest-api.captide.co/api/v1/rag/agent-query-stream",
        json=payload, headers=HEADERS, timeout=120
    )
    return parse_sse_response(r.text)

Reconciliation formats change across quarters. To handle this, we dynamically learn a stable schema using previous reconciliations as a guide. This avoids rigid templates while maintaining consistency—critical for time-series analysis.

In [18]:
BASE_PROMPT = (
    "Return a single valid JSON object with double-quoted keys and numeric values. The object must represent the reconciliation "
    "from Net Income to Adjusted EBITDA, including all reported line items. Numeric values must be converted to thousands of USD, "
    "and the keys must be the name of each line item, but adapted to more common naming conventions if needed. Do not include words like "
    "'add' or 'less' in the keys. Use positive values for metrics that are added to Net Income in the reconciliation and negative "
    "values for metrics that are subtracted. Output only the JSON object—no commentary or extra text."
)

def build_prompt(prev_keys: List[str]) -> str:
    if not prev_keys:
        return BASE_PROMPT
    joined = ", ".join(f'"{k}"' for k in prev_keys)
    return (
        BASE_PROMPT +
        f" Try to fit the values into the following keys if they appear (in this order): [{joined}]. Be flexible but accurate. "
        "If the document contains additional reconciliation line items, insert them at the correct position relative to the list above."
    )

def merge_key_lists(master: list[str], this_quarter: list[str]) -> list[str]:
    for i, k in enumerate(this_quarter):
        if k in master:
            continue
        insert_pos = None
        for j in range(i - 1, -1, -1):
            prev_key = this_quarter[j]
            if prev_key in master:
                insert_pos = master.index(prev_key) + 1
                break
        if insert_pos is None:
            for j in range(i + 1, len(this_quarter)):
                nxt_key = this_quarter[j]
                if nxt_key in master:
                    insert_pos = master.index(nxt_key)
                    break
        if insert_pos is None:
            insert_pos = len(master)
        master.insert(insert_pos, k)
    return master

Using ```run_one_ticker```, we batch process filings for each company, normalize the results, and align the schema. This creates a per-ticker dictionary of clean, time-indexed financial data—ready for modeling or dashboards.

In [19]:
def fiscal_sort_key(fp: str) -> tuple[int, int]:
    m = re.match(r"Q([1-4]) (\d{4})", fp)
    if not m:
        return (9999, 9)
    q, yr = int(m.group(1)), int(m.group(2))
    return (yr, q)

def run_one_ticker(ticker: str) -> Dict[str, Dict[str, float]]:
    docs = fetch_documents(ticker)
    docs.sort(key=lambda d: fiscal_sort_key(d["fiscalPeriod"]))

    key_order: List[str] = []
    results: Dict[str, Dict[str, float]] = {}

    for doc in docs:
        prompt = build_prompt(key_order)
        data = fetch_metrics_with_prompt([doc["sourceLink"]], prompt)
        if not data:
            continue
        results[doc["fiscalPeriod"]] = data
        key_order = merge_key_lists(key_order, list(data.keys()))

    return {"keys": key_order, "data": results}

In [20]:
from concurrent.futures import ThreadPoolExecutor, as_completed

per_ticker_output = {}
with ThreadPoolExecutor(max_workers=len(TICKERS)) as pool:
    futures = {pool.submit(run_one_ticker, t): t for t in TICKERS}
    for fut in as_completed(futures):
        ticker = futures[fut]
        per_ticker_output[ticker] = fut.result()

We convert each company’s results into a tidy pandas.DataFrame:

In [21]:
from IPython.display import display

tables = {}
for ticker, payload in per_ticker_output.items():
    key_order = payload["keys"]
    series_by_q = payload["data"]
    df = pd.DataFrame(series_by_q).reindex(key_order)
    df.index.name = "Line item"
    tables[ticker] = df

for t, frame in tables.items():
    print(f"Reconciliation o {t}")
    display(frame)

Reconciliation o LYFT


,Q1 2023,Q2 2023,Q3 2023,Q4 2023,Q1 2024,Q2 2024,Q3 2024,Q4 2024,Q1 2025
Line item,,,,,,,,,
Net Income,-187600.0,-114300.0,-12100.0,-26300.0,-31500.0,5014.0,-12400.0,22784.0,2600.0
Interest Expense,5900.0,6900.0,7300.0,9700.0,8500.0,9400.0,8900.0,34700.0,7500.0
"Other Income (Expense), Net",-37200.0,-53100.0,-34400.0,-45400.0,-41100.0,-41900.0,-50900.0,-173200.0,-40900.0
Income Tax Provision (Benefit),2700.0,2700.0,100.0,3200.0,2600.0,1900.0,-700.0,2600.0,3400.0
Depreciation and Amortization,27200.0,28600.0,29500.0,31200.0,32400.0,37700.0,45100.0,148900.0,33600.0
Stock-Based Compensation,180400.0,113900.0,98500.0,91700.0,80100.0,85700.0,89000.0,330900.0,93200.0
Payroll Tax Related to Stock-Based Compensation,6200.0,2700.0,1900.0,1600.0,7400.0,4200.0,1700.0,14800.0,4000.0
Net Claims Ceded under Reinsurance Agreement,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
Sublease Income,1300.0,1300.0,1200.0,1100.0,1100.0,1000.0,900.0,3500.0,100.0


Reconciliation o PLTR


,Q1 2023,Q2 2023,Q3 2023,Q4 2023,Q1 2024,Q2 2024,Q3 2024,Q4 2024,Q1 2025
Line item,,,,,,,,,
Net income attributable to common stockholders,16802,28127,71505,93391,105530.0,134126.0,143525.0,79009.0,214031.0
Net income attributable to noncontrolling interests,2349,-255,1934,3522,541.0,1444.0,5816.0,-2073.0,3686.0
Interest income,-20853,-30310,-36864,-44545,-43352.0,-46593.0,-52120.0,-54727.0,-50441.0
Interest expense,1275,1317,742,136,NaN,NaN,NaN,NaN,NaN
"Other income (expense), net",2861,9024,-3864,3956,13507.0,11173.0,8110.0,-14768.0,3173.0
Provision for income taxes,1681,2171,6530,9334,4655.0,5189.0,7809.0,3602.0,5599.0
Depreciation and amortization,8320,8399,8663,7972,8438.0,8056.0,8087.0,7006.0,6622.0
Stock-based compensation,114714,114201,114380,132608,125651.0,141764.0,142425.0,281798.0,155339.0
Employer payroll taxes related to stock-based compensation,6285,10760,8909,10953,19926.0,6464.0,19950.0,79681.0,59323.0


Reconciliation o UBER


,Q1 2023,Q2 2023,Q3 2023,Q1 2024,Q2 2024,Q4 2024,Q1 2025
Line item,,,,,,,
"Net income attributable to Uber Technologies, Inc.",-157000,394000,221000,-654000,1015000,6883000,1776000
"Net income attributable to non-controlling interests, net of tax",0,0,-2000,-9000,-7000,18000,-2000
Provision for income taxes,55000,65000,-40000,29000,57000,-6002000,-402000
Income from equity method investments,-36000,-4000,-3000,4000,12000,10000,-13000
Interest expense,168000,144000,166000,124000,139000,117000,105000
"Other income (expense), net",-292000,-273000,52000,678000,-420000,-256000,-262000
Depreciation and amortization,207000,208000,205000,190000,173000,169000,171000
Stock-based compensation expense,470000,504000,492000,484000,455000,419000,435000
"Legal, tax, and regulatory reserve changes and settlements",250000,-155000,-13000,527000,134000,462000,28000
